## Extract Data

In [ ]:
!unzip '10_classes.zip'

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: 463/2397767.txt         
  inflating: 463/1991169.txt         
  inflating: 463/473083.txt          
  inflating: 463/1657078.txt         
  inflating: 463/2455398.txt         
  inflating: 463/1167187.txt         
  inflating: 463/2235101.txt         
  inflating: 463/2424547.txt         
  inflating: 463/2232091.txt         
  inflating: 463/1031469.txt         
  inflating: 463/245936.txt          
  inflating: 463/2238609.txt         
  inflating: 463/2701783.txt         
  inflating: 463/2262849.txt         
  inflating: 463/2312229.txt         
  inflating: 463/1067342.txt         
  inflating: 463/2049829.txt         
  inflating: 463/828333.txt          
  inflating: 463/2558287.txt         
  inflating: 463/831833.txt          
  inflating: 463/2310438.txt         
  inflating: 463/828975.txt          
  inflating: 463/529353.txt          
  inflating: 463/935016.txt          
  inflating: 463/2509403.txt 

## Function to cleaning data

In [ ]:
import regex as re

def remove_string_special_characters(s, stopwords):
    stripped = re.sub(r'[\W]', ' ', s)
    stripped = re.sub(r'_', ' ', stripped)
    stripped = re.sub(r'[^\D]', ' ', stripped)
    stripped = re.sub(r'ª|º', ' ', stripped)
    for w in stopwords:
      stripped = re.sub(r'\s+'+w+'\s+', ' ', stripped)
    stripped = re.sub(r'\s+', ' ', stripped)
    stripped = stripped.strip()

    return stripped

## Loading Data

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

data = []
label = []
classes = [196,198,200,219,220,339,458,461,463,785]
for c in classes:
    exemplos = os.listdir('./'+str(c))
    for e in exemplos:
        with open('./'+str(c) + '/' + str(e), 'r') as file:
            data.append(file.read().replace('\n', ' '))
            label.append(c)
data[0]

## Download Stopwords of NLTK

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Cleaning texts

In [ ]:
#Removendo stop words
exemplos_semstopword = []
stop_words = nltk.corpus.stopwords.words('portuguese')
stop_words.extend(['aguiar', 'ana', 'andrade', 'araujo', 'araújo', 'bezerra', 'christina', 'duque', 'eduardo', 
                   'fernandes', 'ferreira', 'filho', 'guilherme', 'josé', 'joão', 'junior', 'júnior', 'lauro', 
                   'lima', 'lucena', 'maia', 'maria', 'medeiros', 'oliveira', 'paulo', 'pereira', 'pinheiro', 'pinto', 
                   'pires', 'renata', 'ribeira', 'santos', 'silva', 'souza', 'tais', 'teixeira'])

for exemplo in data:
    exemplos_semstopword.append(remove_string_special_characters(exemplo.lower(), stop_words))

data = exemplos_semstopword.copy()
del exemplos_semstopword

In [ ]:
data[0]

'poder judiciário estado rio grande norte rua doutor candelária natal rn cep processo n ação embargos execução autor embargante crystianne grandi damocles pantaleao lopes trinta réu embargado banco mercantil brasil sa s n t n ç vistos etc tratam autos embargos execução ajuizados crystiane grandi damocles pantaleão lopes trinta face banco mercantil brasil s a sr chefe secretaria certificou id presentes embargos ajuizados intempestivamente relatório decido embargos execução previstos legislação processual via adequada possibilitar discussão dívida sob enfoque executado oportunizando apontar vícios título executivo opor obrigação cobrada liquidação ainda parcial débito caso presente porém possível apreciar presentes embargos serem intempestivos respeito dispõe art cpc embargos oferecidos prazo quinze dias contados data juntada autos mandado citação todavia mandados citatórios juntados autos dias dâmocles pantaleão crystianne grandi enquanto presentes embargos ajuizados posto rejeito limin

In [ ]:
label = np.array(label)

treino, teste, treino_y, teste_y = train_test_split(data, label, test_size=0.2, shuffle=True, random_state=0)

## Make TFidf transformer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_transformer = TfidfVectorizer(min_df = 0.05, max_df = 0.90)
X_train_tfidf = tfidf_transformer.fit_transform(treino)
X_test_tfidf = tfidf_transformer.transform(teste)

In [ ]:
X_train = [b.tolist() for b in tfidf_transformer.inverse_transform(X_train_tfidf)]
X_train = [' '.join([b for b in i]) for i in X_train]
X_train

['forma digitalmente assinado documento juiz novembro intime cite conciliação audiência consoante constante formulado defiro exposto acima promovente pleito necessário assim fica ser deve obrigação portanto objeto endereço consta contrato proferida vista tendo autor pretensão acolhimento merece cpc prevista artigo dispõe conforme mérito resolução extinção casos magistrado civil código novo possibilidade ante pedido apenas inicialmente desta motivo sendo nova município demanda deu juízo feito eis material erro existência síntese alegando contra id declaração embargos etc vistos sentença sa ré autora cep rua comarca cível especial juizado estado judiciário poder',
 'juíza dezembro natal intimem registre publique advocatícios honorários custas deixo requerimento independentemente execução julgado trânsito após dias dez prazo advogado através lo devem havendo citação desde mora juros data partir monetária correção incidência favor título pagamento condeno procedente julgo dispositivo causa

In [ ]:
X_train_tfidf.shape

(24000, 850)

In [ ]:
import pandas as pd
tds_tfidf = []
X_test = [b.tolist() for b in tfidf_transformer.inverse_transform(X_test_tfidf)]
X_test = [' '.join([b for b in i]) for i in X_test]
tds_tfidf.extend(X_train)
tds_tfidf.extend(X_test)

y_tfidf = []
y_tfidf.extend(treino_y)
y_tfidf.extend(teste_y)
df = pd.DataFrame()
df['texts'] = tds_tfidf
df['label'] = y_tfidf
df.to_csv('./data_clear.csv')

In [ ]:
tfidf_transformer.get_feature_names()

['abril',
 'ac',
 'acerca',
 'acima',
 'acolhimento',
 'acolho',
 'acordo',
 'acrescido',
 'acórdão',
 'ademais',
 'adequada',
 'adimplemento',
 'administração',
 'admissibilidade',
 'aduz',
 'advocatícios',
 'advogado',
 'afirma',
 'agir',
 'agosto',
 'agravo',
 'agrg',
 'aguarde',
 'ainda',
 'ajuizamento',
 'ajuizou',
 'alameda',
 'alega',
 'alegado',
 'alegados',
 'alegando',
 'alegação',
 'alegações',
 'alguns',
 'alvará',
 'além',
 'ambos',
 'analisando',
 'analisar',
 'andar',
 'anexo',
 'ano',
 'anos',
 'ante',
 'antecipada',
 'antecipação',
 'anterior',
 'anteriormente',
 'antes',
 'antiga',
 'análise',
 'apelação',
 'apenas',
 'apesar',
 'aplicação',
 'aplicável',
 'aprazada',
 'apreciação',
 'apresenta',
 'apresentados',
 'apresentar',
 'apresentou',
 'após',
 'aqui',
 'argumento',
 'argumentos',
 'arquivamento',
 'arquive',
 'arquivem',
 'art',
 'artigo',
 'artigos',
 'arts',
 'assim',
 'assinado',
 'assiste',
 'assistência',
 'ativa',
 'ato',
 'atos',
 'através',
 'audiênci

In [ ]:
vocabulary = pd.DataFrame(tfidf_transformer.get_feature_names())
vocabulary.to_csv('./vocabulary.csv')